In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb
import io
import yfinance as yf

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn import metrics
from sklearn.linear_model import LinearRegression
import zipfile
from sklearn.metrics import confusion_matrix

import warnings
warnings.filterwarnings('ignore')

C:\Users\mihir\anaconda3\Lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
 with open('data/VOO.csv', 'r') as file:
    VOO = pd.read_csv(file)
    

In [3]:
VOO.describe()

,Close/Last,Volume,Open,High,Low
count,2516.000000,2.516000e+03,2516.000000,2516.000000,2516.000000
mean,298.802294,3.571020e+06,298.761434,300.336076,297.035737
std,93.734034,2.494741e+06,93.717454,94.226332,93.168078
min,167.700000,4.302330e+05,167.260000,168.720000,165.960000
25%,217.055000,1.987620e+06,216.840000,217.829500,215.860000
50%,269.180000,3.017750e+06,269.060000,270.200000,267.960000
75%,379.702500,4.384720e+06,379.562500,382.092500,377.350000
max,527.700000,3.541805e+07,529.670000,529.920000,525.490000


In [4]:
VOO["Close"] = VOO["Close/Last"]
VOO['open-close']  = VOO['Open'] - VOO['Close']
VOO['low-high']  = VOO['Low'] - VOO['High']
VOO['target'] = np.where(VOO['Close'].shift(-1) > VOO['Close'], 1, 0)
VOO['Date'] = pd.to_datetime(VOO['Date'], format='%m/%d/%Y')

# Extract month, day, and year into separate columns
VOO['month'] = VOO['Date'].dt.month
VOO['day'] = VOO['Date'].dt.day
VOO['year'] = VOO['Date'].dt.year
VOO['is_quarter_end'] = np.where(VOO['month']%3==0,1,0)

In [5]:
VOO

,Date,Close/Last,Volume,Open,High,Low,Close,open-close,low-high,target,month,day,year,is_quarter_end
0,2024-09-30,527.67,8017349,524.32,527.9550,522.19,527.67,-3.35,-5.7650,0,9,30,2024,1
1,2024-09-27,525.38,5598594,527.09,527.7700,524.32,525.38,1.71,-3.4500,1,9,27,2024,1
2,2024-09-26,527.70,5273915,529.67,529.9200,525.49,527.70,1.97,-4.4300,0,9,26,2024,1
3,2024-09-25,525.61,35418050,526.61,527.3000,524.65,525.61,1.00,-2.6500,1,9,25,2024,1
4,2024-09-24,526.68,4650608,526.01,526.8400,523.38,526.68,-0.67,-3.4600,0,9,24,2024,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2511,2014-10-07,177.22,930590,179.05,179.4500,177.20,177.22,1.83,-2.2500,1,10,7,2014,0
2512,2014-10-06,179.98,1306964,180.94,181.1807,179.36,179.98,0.96,-1.8207,1,10,6,2014,0
2513,2014-10-03,180.20,1217352,179.48,180.5500,178.88,180.20,-0.72,-1.6700,0,10,3,2014,0
2514,2014-10-02,178.21,1263791,178.05,178.8300,176.40,178.21,-0.16,-2.4300,1,10,2,2014,0


In [6]:
VOO["year"].unique()

array([2024, 2023, 2022, 2021, 2020, 2019, 2018, 2017, 2016, 2015, 2014])

In [22]:
features = VOO[['open-close', 'low-high', 'is_quarter_end' ]]
target = VOO['target']
 
scaler = StandardScaler()
features = scaler.fit_transform(features)
 
train_VOO = VOO[VOO['year'] <= 2019]  # Training data
test_VOO = VOO[VOO['year'] >= 2022]    # Testing data

# Select features and target variable
X_train = train_VOO[['open-close', 'low-high', 'is_quarter_end' ]]  
y_train = train_VOO['target']
X_test = test_VOO[['open-close', 'low-high', 'is_quarter_end' ]]    
y_test = test_VOO['target']



In [23]:
models = [ LogisticRegression(), SVC(
  kernel='poly', probability=True), XGBClassifier()]
for i in range(3):
      models[i].fit(X_train, y_train)

      print(f'{models[i]} : ')
      print('Training Accuracy : ', metrics.roc_auc_score(
        y_train, models[i].predict_proba(X_train)[:,1]))
      print('Validation Accuracy : ', metrics.roc_auc_score(
        y_test, models[i].predict_proba(X_test)[:,1]))
      print()

LogisticRegression() : 
Training Accuracy :  0.866391622438449
Validation Accuracy :  0.8881601508916325

SVC(kernel='poly', probability=True) : 
Training Accuracy :  0.8398500847707504
Validation Accuracy :  0.8636316872427984

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...) : 
Training A

In [19]:
for lag in range(1, 6):  # Lag_1 to Lag_5
    VOO[f'Lag_{lag}'] = VOO['Close'].shift(lag)
VOO = VOO.dropna().reset_index(drop=True)

In [20]:
VOO

,Date,Close/Last,Volume,Open,High,Low,Close,open-close,low-high,target,month,day,year,is_quarter_end,Lag_1,Lag_2,Lag_3,Lag_4,Lag_5
0,2024-09-23,525.17,29289160,525.00,525.8300,523.83,525.17,-0.17,-2.0000,0,9,23,2024,1,526.68,525.61,527.70,525.38,527.67
1,2024-09-20,523.88,5612678,523.51,524.9200,521.12,523.88,-0.37,-3.8000,1,9,20,2024,1,525.17,526.68,525.61,527.70,525.38
2,2024-09-19,524.91,5558371,524.88,526.6000,522.19,524.91,-0.03,-4.4100,0,9,19,2024,1,523.88,525.17,526.68,525.61,527.70
3,2024-09-18,515.91,6921278,518.18,522.7300,515.47,515.91,2.27,-7.2600,1,9,18,2024,1,524.91,523.88,525.17,526.68,525.61
4,2024-09-17,517.59,8149957,519.44,520.7350,515.50,517.59,1.85,-5.2350,0,9,17,2024,1,515.91,524.91,523.88,525.17,526.68
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2506,2014-10-07,177.22,930590,179.05,179.4500,177.20,177.22,1.83,-2.2500,1,10,7,2014,0,180.32,176.68,174.70,171.83,172.11
2507,2014-10-06,179.98,1306964,180.94,181.1807,179.36,179.98,0.96,-1.8207,1,10,6,2014,0,177.22,180.32,176.68,174.70,171.83
2508,2014-10-03,180.20,1217352,179.48,180.5500,178.88,180.20,-0.72,-1.6700,0,10,3,2014,0,179.98,177.22,180.32,176.68,174.70
2509,2014-10-02,178.21,1263791,178.05,178.8300,176.40,178.21,-0.16,-2.4300,1,10,2,2014,0,180.20,179.98,177.22,180.32,176.68


In [25]:
last_row = test_VOO.iloc[-1].copy()



model_names = [
    "Logistic_Regression",
    "SVC_Polynomial",
    "XGBoost_Classifier"
]
results = []
for lag in range(1, 6):
    if f'Lag_{lag}' not in last_row:
        last_row[f'Lag_{lag}'] = last_row['Close'] 

for model, name in zip(models, model_names):
    last_row = test_VOO.iloc[-1].copy()
    predicted_directions = []

    for _ in range(60):
        features = last_row[['open-close', 'low-high', 'is_quarter_end']].values.reshape(1, -1)
        prob_up = model.predict_proba(features)[0][1]
        predicted_directions.append(prob_up)

        
        predicted_price = last_row['Close'] * (1 + (0.01 if prob_up > 0.5 else -0.005))
        last_row['Close'] = predicted_price

        
        for lag in range(5, 0, -1):
            last_row[f'Lag_{lag}'] = last_row[f'Lag_{lag-1}'] if lag != 1 else predicted_price

        
        last_row['open-close'] = last_row['Open'] - last_row['Close']
        last_row['low-high'] = last_row['Low'] - last_row['High']
        future_dates = pd.date_range(
            start=test_VOO.iloc[-1]['Date'] + pd.Timedelta(days=1),
            periods=60,
            freq='B'
        )

    df = pd.DataFrame({
        'Date': future_dates,
        f'{name}_Prob_Up': predicted_directions
    })

    results.append(df)

final_predictions = results[0]
for df in results[1:]:
    final_predictions = final_predictions.merge(df, on='Date')

print(final_predictions)


         Date  Logistic_Regression_Prob_Up  SVC_Polynomial_Prob_Up  \
0  2022-01-04                     0.144887                0.343675   
1  2022-01-05                     0.879124                0.714182   
2  2022-01-06                     0.004082                0.300721   
3  2022-01-07                     0.151990                0.343571   
4  2022-01-10                     0.884942                0.726689   
5  2022-01-11                     0.004318                0.304834   
6  2022-01-12                     0.159376                0.343536   
7  2022-01-13                     0.890514                0.739150   
8  2022-01-14                     0.004568                0.308782   
9  2022-01-17                     0.167049                0.343574   
10 2022-01-18                     0.895848                0.751531   
11 2022-01-19                     0.004832                0.312566   
12 2022-01-20                     0.175015                0.343690   
13 2022-01-21       

In [26]:
final_predictions.to_csv("predicted_probabilities.csv", index=False)